# Temperatura

Para ver el concepto de Temperatura y Cambio Climático revise la [Introduccion](Introduction.ipynb).
+ Mapas de Temperatura Media de años recientes de cada departamento.
+ Gráfico de Tendencia de la Temperatura Promedio de Colombia.

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Mapas de Temperatura Media

In [2]:
#Se descargan las librerias necesarias 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy as cart
import scipy.stats #Metodos estadisticos
import matplotlib.colors as colors #Set Colors
import wradlib.ipol as ipol #Metodo de Interpolación

# El metodo para aplicar gradiente vertical de temperatura
def regresion(zi, altitude, nivel, coeficiente):
    temperature = []
    for ZI_item, altitude_item in zip(zi, altitude):
        if str(nivel) == 'altura_base':
            ZI_item = ZI_item - coeficiente*altitude_item
        elif str(nivel) == 'altura_real': 
            ZI_item = ZI_item + coeficiente*altitude_item
        #agregar elemento dentro de temperatura
        temperature.append(ZI_item)
    #se devuelve el valor de temperatura 
    return temperature 

class Mapa:  
    # El metodo inicial o constructor
    def __init__(self, departamento, year, mes, tipo):
     
        # Instance Variable    
        self.departamento = departamento
        self.year = year
        self.mes = mes
        self.tipo = tipo
        self.normal = '1981-2010' 
        
    # El metodo para contruir mapas     
    def get_mapa(self, ax, periodo):
        
        #Se define la extensión del mapa
        if str(self.departamento) == 'Santander':
            lllat, urlat, lllon, urlon = 5.5, 8.5, -75, -72   
        elif str(self.departamento) == 'Magdalena':
            lllat, urlat, lllon, urlon = 8.7, 11.5, -75.55, -73
    
        #Se establece instancia de Cartopy    
        ax.set_extent([lllon, urlon, lllat, urlat], crs=cart.crs.PlateCarree())
        ax.add_feature(cart.feature.OCEAN, zorder=100, edgecolor='k')#, facecolor = 'white')
    
        #Se dibujan los departamentos de Colombia
        fname = r".\DIVISION_COLOMBIA\MGN_DPTO_POLITICO.shp"
        adm1_shapes = list(cart.io.shapereader.Reader(fname).geometries())
        ax.add_geometries(adm1_shapes, cart.crs.PlateCarree(), edgecolor='black', facecolor='none', alpha=0.5)

        #Se cargan los datos de las estaciones    
        df = pd.read_csv(str(r".\Datos\Temperatura\Temperatura_"+str(periodo)+".csv"))

        #Se seleccionan las estaciones que se ubiquen en el area del mapa 
        data_magdalena = df.loc[(df['LONGITUD'] < urlon) & (df['LONGITUD'] > lllon)
                  & (df['LATITUD'] < urlat) & (df['LATITUD'] > lllat)]

        #Se remueven filas de dataframe que no contienen valores para el mes especifico
        data_magdalena = data_magdalena.dropna(subset=[str(self.mes)])

        #Se extrae los valores de coordenadas, altura y valor de cada estacion
        longitud = data_magdalena['LONGITUD'].tolist()
        latitud = data_magdalena['LATITUD'].tolist()
        elevacion = data_magdalena['ELEVACION'].tolist()
        valor = data_magdalena[str(self.mes)].tolist()
        src = np.vstack( (np.array(longitud), np.array(latitud)) ).transpose()

        #Se crea una grilla 300x300
        numcols, numrows = 350, 350
        xtrg = np.linspace(lllon, urlon, numcols)
        ytrg = np.linspace(lllat, urlat, numrows)
        trg = np.meshgrid(xtrg, ytrg)
        trg = np.vstack( (trg[0].ravel(), trg[1].ravel()) ).T

        #Se calcula el coeficiente de regresión entre altura y temperatura a partir de los datos de las estaciones 
        result = scipy.stats.linregress(np.array(elevacion), np.array(valor))
        coeficiente, r = result.slope, result.rvalue 
        if self.departamento == 'Magdalena' and r > -0.46:
            dg = pd.read_csv(str(r".\Regresion de Elevacion\Regresion_Temperatura_Magdalena_2018.csv"))
            coeficiente = dg[str(self.mes)][0]      
        z = regresion(valor, elevacion, "altura_base", coeficiente)
    
        #Se aplica el método de interpolación OK (Ordinary Kriging)
        idw = ipol.Idw(src, trg)
        zi = idw(np.array(z))

        #Regresion a la altura Real
        data_hh = pd.read_csv(str(r".\Regresion de Elevacion\Elevacion_"+str(self.departamento)+".csv"))
        altitude = np.array(data_hh["Elevacion"].tolist())
        ZI = np.array(regresion(zi, altitude, "altura_real",coeficiente))

        if str(periodo) == "1981-2010":
            self.Normal, self.x, self.y = ZI, xtrg, ytrg
        elif str(periodo) != "1981-2010" and str(self.tipo) == "Anomalia":
            return ZI, ax
        
        #Se dibuja los valores en el mapa
        cmap = plt.get_cmap('jet',50)
        con = plt.pcolormesh(xtrg, ytrg, ZI.reshape( (numcols, numrows) ), transform=cart.crs.PlateCarree(), alpha=0.6, cmap = cmap, vmin = 9, vmax = 34)
    
        #Se dibuja la grafica de colores
        cbar = plt.colorbar(con, orientation='horizontal',fraction=.042, pad=0.03)
        
    def plot_mapas(self):
        # Se define el tamaño del cuadro 
        plt.figure(figsize=(12.4, 9.4))

        # Tabla 1x3 y dibujaremos en la celda 1
        ax = plt.subplot(1, 2, 1, projection=cart.crs.PlateCarree())
        ax.set_title('Normal Climatologica Temperatura (1981-2010)',fontdict = {'fontsize':13})
        self.get_mapa(ax,self.normal)
        normal, x, y = self.Normal, self.x, self.y

        # Tabla 1x3 y dibujaremos en la celda 2
        ax = plt.subplot(1, 2, 2, projection=cart.crs.PlateCarree()) 
        ax.set_title('Temperatura Media '+str(self.year),fontdict = {'fontsize':13})    
    
        #Se obtiene el mapa de Anomalia de Temperatura
        if str(self.tipo) == 'Anomalia':
            actual, ax = self.get_mapa(ax,self.year) 
            anomalia = np.subtract(actual,normal)
            cmap = colors.ListedColormap(["#0000ff","#0170fe","#00c5ff","#bee8ff","#fefffe","#ffebaf","#ffd37f","#feab00","#fe5501"])
            levels = [-5.0, -2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0, 5.0]
            norm = colors.BoundaryNorm(levels, cmap.N)
            con = plt.pcolormesh(x,y,anomalia.reshape( (350, 350) ), transform=cart.crs.PlateCarree(), alpha=0.6, cmap = cmap, norm = norm)
            cbar = plt.colorbar(con, orientation='horizontal',fraction=.042, pad=0.03)
            ax.set_title('Anomalia de Temperatura Media '+str(self.year),fontdict = {'fontsize':13})
        else:
            self.get_mapa(ax,self.year)
    
        # Dibujamos el conjunto
        plt.show()

In [3]:
import ipywidgets as widgets
import time

a = widgets.Dropdown(options=[('Enero', 'ENE'), ('Febrero', 'FEB'), ('Marzo', 'MAR'), ('Abril', 'ABR'), ('Mayo', 'MAY'),
                             ('Junio', 'JUN'), ('Julio', 'JUL'), ('Agosto', 'AGO'), ('Septiembre', 'SEP'),
                             ('Noviembre', 'NOV'), ('Diciembre', 'DIC')], description='Mes:')
b = widgets.Dropdown(options=[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021], description='Año:')
c = widgets.Dropdown(options=['Magdalena', 'Santander'], description='Dep:')
d = widgets.Dropdown(options=[('Anomalía', 'Anomalia'), ('Valor Absoluto', 'Absoluto')], description='Tipo:')
ui = widgets.HBox([a, b, c, d])
def f(a, b, c, d):
    start_time = time.time()
    m = Mapa(c,b,a,d)
    m.plot_mapas()
    print("--- %s seconds ---" % (time.time() - start_time))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c, 'd': d})

display(ui, out)

Output()

## Gráfico de Tendencia Nacional

In [7]:
import plotly.express as px
import pandas as pd
import numpy as np
import datetime as dt

def get_trend(tipo):
    #Se importa los datos
    df = pd.read_csv(str(r".\Datos\Temperatura\Temperatura_Nacional.csv"))
    df.Fecha = pd.to_datetime(df['Fecha'], format = '%d/%m/%Y')

    if tipo == 'Anual':
        a,time = [],[]
        for year in range(1984,2021):
            filtro_df = df.loc[(df['Fecha'].dt.year == year)]
            promedio = np.mean(filtro_df['Temperatura del Aire'])
            a.append(promedio)
            time.append(dt.date(year,1,1))    
        df = pd.DataFrame({'Fecha': time, 'Temperatura del Aire': a, 'Normal Climatica (1980-2010)': df['Normal Climatica (1980-2010)'][0]})
        
    fig = px.line(df, x="Fecha", y=["Normal Climatica (1980-2010)", "Temperatura del Aire"])
    fig.update_traces(mode="markers+lines", hovertemplate=None)
    fig.update_layout(hovermode="x unified", yaxis_title="°C (Grado Centígrado)", xaxis_title="Tiempo",
                     title_text="Temperatura Media de Colombia 1984-2021", title_x=0.22)
    fig.show()

In [8]:
import ipywidgets as widgets

a = widgets.Dropdown(options=['Mensual', 'Anual'], description='Tipo:')
ui = widgets.HBox([a])
def f(a):
    get_trend(a)    

out = widgets.interactive_output(f, {'a': a})

display(ui, out)

Output()